<a href="https://colab.research.google.com/github/Anna671/NLP/blob/main/NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
#Importing libraries 
import pandas as pd
import numpy as np
import tensorflow as tf
import keras.backend as K
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding
from keras.layers import LSTM, GRU, SimpleRNN
from keras.datasets import imdb

In [4]:
from keras.callbacks import History

In [5]:
history = History()

In [6]:
df = pd.read_csv('/content/train.csv')

In [7]:
df

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [8]:
#Deleting columns
df.drop(['keyword', 'location'], axis=1, inplace=True)

In [9]:
df

,id,text,target
0,1,Our Deeds are the Reason of this #earthquake M...,1
1,4,Forest fire near La Ronge Sask. Canada,1
2,5,All residents asked to 'shelter in place' are ...,1
3,6,"13,000 people receive #wildfires evacuation or...",1
4,7,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...
7608,10869,Two giant cranes holding a bridge collapse int...,1
7609,10870,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,Police investigating after an e-bike collided ...,1


In [10]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(
                                    df['text'], df['target'], test_size=0.33, random_state=42)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((5100,), (2513,), (5100,), (2513,))

In [11]:
def max_lenth(max_l):
    max_l = 150
    for i in x_train:
        if len(i) > max_l: max_l = len(i)
    return max_l
max_l = max_lenth(df['text'])
max_l

157

In [12]:
# Find the maximum length of any caption in our dataset
def calc_max_length(tensor):
    return max(len(t) for t in tensor)

In [16]:
# Choose the top 700 words from the vocabulary
top_k = 700
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=top_k,
                                                  oov_token="<unk>",
                                                  filters='!"#$%&()*+.,-/:;=?@[\]^_`{|}~ ')
tokenizer.fit_on_texts(x_train)

In [18]:
# Create the tokenized vectors
x_train_seqs = tokenizer.texts_to_sequences(x_train)
X_train = tf.keras.preprocessing.sequence.pad_sequences(x_train_seqs, maxlen=max_l)

In [19]:
# Pad each vector to the max_length of the captions
# If you do not provide a max_length value, pad_sequences calculates it automatically
x_test_seqs = tokenizer.texts_to_sequences(x_test)
X_test = tf.keras.preprocessing.sequence.pad_sequences(x_test_seqs, maxlen=max_l)

In [20]:
X_train.shape,y_train.shape

((5100, 157), (5100,))

In [21]:
import tensorflow as tf
import keras.backend as K
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding
from keras.layers import LSTM, GRU, SimpleRNN
from keras.datasets import imdb

In [22]:
model = Sequential()

model.add(Embedding(top_k, 700))
model.add(LSTM(50, dropout = 0.3))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy', 'AUC'])

In [23]:
#Training

%%time
history = model.fit(X_train, y_train, 
                    batch_size = 100,
                    epochs = 20,
                    validation_data = (X_test, y_test),
                    verbose = 1, callbacks=[history]   
                    )

Epoch 1/20
51/51 [==============================] - 56s 738ms/step - loss: 0.6539 - accuracy: 0.6229 - auc: 0.6494 - val_loss: 0.5474 - val_accuracy: 0.7425 - val_auc: 0.8147
Epoch 2/20
51/51 [==============================] - 35s 692ms/step - loss: 0.4555 - accuracy: 0.7909 - auc: 0.8593 - val_loss: 0.4810 - val_accuracy: 0.7728 - val_auc: 0.8362
Epoch 3/20
51/51 [==============================] - 37s 729ms/step - loss: 0.4027 - accuracy: 0.8184 - auc: 0.8915 - val_loss: 0.4758 - val_accuracy: 0.7863 - val_auc: 0.8385
Epoch 4/20
51/51 [==============================] - 37s 736ms/step - loss: 0.3719 - accuracy: 0.8412 - auc: 0.9047 - val_loss: 0.4803 - val_accuracy: 0.7915 - val_auc: 0.8400
Epoch 5/20
51/51 [==============================] - 37s 725ms/step - loss: 0.3564 - accuracy: 0.8456 - auc: 0.9132 - val_loss: 0.4981 - val_accuracy: 0.7784 - val_auc: 0.8386
Epoch 6/20
51/51 [==============================] - 37s 732ms/step - loss: 0.3347 - accuracy: 0.8568 - auc: 0.9241 - val_loss

In [24]:
model.evaluate(X_test,y_test,batch_size=5)

503/503 [==============================] - 7s 15ms/step - loss: 0.9126 - accuracy: 0.7489 - auc: 0.7998


[0.9126424193382263, 0.7489057183265686, 0.7998054623603821]

In [25]:
y_pred = model.predict(X_test)

In [35]:
y_pred_val = np.round(model.predict(X_valid)).astype(int)
y_pred_val

array([[1],
       [1],
       [0],
       ...,
       [1],
       [1],
       [0]])

## Validation prediction 

In [29]:
df_test = pd.read_csv('/content/test.csv')

In [31]:
df_test

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


In [32]:
x_valid = df_test['text']

In [33]:
x_valid_seqs = tokenizer.texts_to_sequences(x_valid)
X_valid = tf.keras.preprocessing.sequence.pad_sequences(x_valid_seqs, maxlen=max_l)

In [34]:
y_pred_val = np.round(model.predict(X_valid)).astype(int)
y_pred_val

array([[1],
       [1],
       [0],
       ...,
       [1],
       [1],
       [0]])

In [37]:
df_test['target']=y_pred_val
df_test[['id','target']].to_csv('sample_submission7.csv',index=False)